# Memory & Strides

## 1. Problem Statement: The Transpose 
You are given a 2D tensor $M$ of shape (3, 2). 
1. Manually define its "Storage" (the flat array in memory).
2. Change its shape without moving the numbers in memory.
3. Transpose it and explain why PyTorch says the data hasn't moved, even though the shape changed.

### The Setup
```python
# !pip install -q matplotlib
import torch
import numpy as np

# Create a 3x2 tensor
# [ 1, 2 ]
# [ 3, 4 ]
# [ 5, 6 ]
M = torch.tensor([[1, 2], [3, 4], [5, 6]])
```

### Your Challenge:
1. **The Metadata:** Access `M.storage()`, `M.stride()`, and `M.storage_offset()`.
2. **The Prediction:** If I call `M_t = M.t()` (transpose), what will be the new `stride`? 
3. **The Proof:** Modify a single value in `M`. Does it change in `M_t`? Why?
4. **The "Contiguous" Trap:** Try to call `M_t.view(-1)`. It will fail. **Why?** And how do you fix it from a first-principle perspective?


<details>
<summary>Hint</summary>

*   **Storage:** The actual 1D array in your RAM: `[1, 2, 3, 4, 5, 6]`.
*   **Stride:**  It tells PyTorch: "To move to the next row, skip $X$ elements. To move to the next column, skip $Y$ elements."
*   **View vs. Reshape:** One is a "Metadata change," the other might "Copy data."

</details>

In [1]:
import torch
import numpy as numpy

M = torch.tensor ([[1,2], [3,4], [5,6]])

In [7]:
#1
print(f" M.storage: {M.storage()}  \n M.stride: {M.stride()} \n M.storage_offset: {M.storage_offset()} \n M.size(): {M.size()}")

 M.storage:  1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]  
 M.stride: (2, 1) 
 M.storage_offset: 0 
 M.size(): torch.Size([3, 2])


In [8]:
#2
M_t = M.t()
print(f"M_t: {M_t} \n M_t.storage: {M_t.storage()}  \n M_t.stride: {M_t.stride()} \n M_t.storage_offset: {M_t.storage_offset()} M_t.size(): {M_t.size()}")

M_t: tensor([[1, 3, 5],
        [2, 4, 6]]) 
 M_t.storage:  1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]  
 M_t.stride: (1, 2) 
 M_t.storage_offset: 0 M_t.size(): torch.Size([2, 3])


In [ ]:
#3
# changing [0,1] element of M
M[0,1] = 20
print(f"After changing M[0,1] to 20: \n M: {M} \n M_t: {M_t}")

After changing M[0,1] to 20: 
 M: tensor([[ 1, 20],
        [ 3,  4],
        [ 5,  6]]) 
 M_t: tensor([[ 1,  3,  5],
        [20,  4,  6]])


In [10]:
#4
print(M_t.view(-1))

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

### Analysis

#### The Stride Math 
output showed:
*   `M.stride()`: `(2, 1)` $\rightarrow$ To move 1 row, jump 2 elements. To move 1 col, jump 1.
*   `M_t.stride()`: `(1, 2)` $\rightarrow$ To move 1 row, jump **1** element. To move 1 col, jump **2**.

 Transposing in PyTorch is **Zero-Latency**. It doesn't move a single byte in RAM. It just swaps the stride numbers. 

#### Why did `.view()` fail?
The `.view()` method has a strict contract: **The logical order must match the physical order.**
*   In `M`, the physical storage is `[1, 2, 3, 4, 5, 6]`. To read it logically, you go `1, 2` (row 1), then `3, 4` (row 2). This matches the storage perfectly. It is **Contiguous**.
*   In `M_t`, the logical order is `[1, 3, 5]` for the first row. But in physical memory, `1` and `3` are not neighbors (`2` is between them). This is **Non-Contiguous**.

**The Fix:** `M_t.contiguous().view(-1)`
`.contiguous()` is the command that says: *"Okay PyTorch, actually move the numbers in RAM so they match my current logical view."*
